In this notebook we implement and make some plots for constraining a NN with SBL, and set the noise level equal on both:

In [1]:
# %% General imports
import numpy as np
import torch

# DeepMoD stuff
from multitaskpinn import DeepMoD
from multitaskpinn.model.func_approx import NN
from multitaskpinn.model.library import Library1D
from multitaskpinn.model.constraint import LeastSquares
from multitaskpinn.model.sparse_estimators import Threshold
from multitaskpinn.training.sparsity_scheduler import TrainTestPeriodic
from multitaskpinn.utils.logger import Logger
from multitaskpinn.training.convergence import Convergence

from deepymod.data import Dataset
from deepymod.data.burgers import BurgersDelta
from deepymod.analysis import load_tensorboard

%load_ext autoreload
%autoreload 2

In [2]:
# Settings
#if torch.cuda.is_available():
#    device = 'cuda'
#else:
#    device = 'cpu'

device = 'cpu'

# Settings for reproducibility
np.random.seed(42)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Making dataset

In [45]:
# Making dataset
v = 0.1
A = 1.0

x = np.linspace(-3, 4, 10)
t = np.linspace(0.1, 5.0, 100)
x_grid, t_grid = np.meshgrid(x, t, indexing='ij')
dataset = Dataset(BurgersDelta, v=v, A=A)
X, y = dataset.create_dataset(x_grid.reshape(-1, 1), t_grid.reshape(-1, 1), n_samples=0, noise=0.2, random=True, normalize=True)
X, y = X.to(device), y.to(device)

# SBL training function

In [41]:
def train_SBL(model: DeepMoD,
          data: torch.Tensor,
          target: torch.Tensor,
          optimizer,
          sparsity_scheduler,
          split = 0.8,
          exp_ID: str = None,
          log_dir: str = None,
          max_iterations: int = 10000,
          write_iterations: int = 25,
          **convergence_kwargs) -> None:
    """Trains the DeepMoD model. This function automatically splits the data set in a train and test set. 

    Args:
        model (DeepMoD):  A DeepMoD object.
        data (torch.Tensor):  Tensor of shape (n_samples x (n_spatial + 1)) containing the coordinates, first column should be the time coordinate.
        target (torch.Tensor): Tensor of shape (n_samples x n_features) containing the target data.
        optimizer ([type]):  Pytorch optimizer.
        sparsity_scheduler ([type]):  Decides when to update the sparsity mask.
        split (float, optional):  Fraction of the train set, by default 0.8.
        exp_ID (str, optional): Unique ID to identify tensorboard file. Not used if log_dir is given, see pytorch documentation.
        log_dir (str, optional): Directory where tensorboard file is written, by default None.
        max_iterations (int, optional): [description]. Max number of epochs , by default 10000.
        write_iterations (int, optional): [description]. Sets how often data is written to tensorboard and checks train loss , by default 25.
    """
    logger = Logger(exp_ID, log_dir)
    sparsity_scheduler.path = logger.log_dir # write checkpoint to same folder as tb output.
    
    # Splitting data, assumes data is already randomized
    n_train = int(split * data.shape[0])
    n_test = data.shape[0] - n_train
    data_train, data_test = torch.split(data, [n_train, n_test], dim=0)
    target_train, target_test = torch.split(target, [n_train, n_test], dim=0)

    n_samples = data_train.shape[0]
    model.t.data = -torch.var(target)
    model.b.data = -torch.var(target)

    # Training
    convergence = Convergence(**convergence_kwargs)
    for iteration in torch.arange(0, max_iterations):
        # ================== Training Model ============================
        prediction, time_derivs, thetas = model(data_train)

        tau_ = torch.exp(model.t).clamp(max=1e8) 
        alpha_ = torch.exp(model.a).clamp(max=1e8)  # we train the log of these things since they're very big
        beta_ = torch.exp(model.b).clamp(max=1e8) 
        
        mask = model.constraint.sparsity_masks[0]
        p_reg, mn = SBL(thetas[0][:, mask] , time_derivs[0], alpha_[mask], tau_.data) # we feed tau but dont differentiate
        p_MSE = tau_ * torch.sum((prediction - target_train)**2, dim=0) - n_samples * torch.log(tau_)
        loss = torch.sum(p_MSE + p_reg)
            
        MSE = torch.mean((prediction - target_train)**2, dim=0)  # loss per output
        Reg = torch.stack([torch.mean((dt - theta @ coeff_vector)**2)
                           for dt, theta, coeff_vector in zip(time_derivs, thetas, model.constraint_coeffs(scaled=False, sparse=True))])

        # Optimizer step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if iteration % write_iterations == 0:
            # ================== Validation costs ================
            with torch.no_grad():
                prediction_test = model.func_approx(data_test)[0]
                MSE_test = torch.mean((prediction_test - target_test)**2, dim=0)  # loss per output
         
            # ====================== Logging =======================
            _ = model.sparse_estimator(thetas, time_derivs) # calculating estimator coeffs but not setting mask
            logger(iteration, 
                   loss, MSE, Reg,
                   model.constraint_coeffs(sparse=True, scaled=True), 
                   model.constraint_coeffs(sparse=True, scaled=False),
                   model.estimator_coeffs(),
                   MSE_test=MSE_test,
                   p_MSE=p_MSE,
                   p_reg=p_reg,
                   tau=tau_,
                   alpha=alpha_,
                   beta_=beta_,
                   mn=mn)

            # ================== Sparsity update =============
            # Updating sparsity 
            update_sparsity = sparsity_scheduler(iteration, torch.sum(MSE_test), model, optimizer)
            if update_sparsity: 
                model.constraint.sparsity_masks = model.sparse_estimator(thetas, time_derivs)

            # ================= Checking convergence
            l1_norm = torch.sum(torch.abs(torch.cat(model.constraint_coeffs(sparse=True, scaled=True), dim=1)))
            converged = convergence(iteration, l1_norm)
            if converged:
                break
    logger.close(model)

In [46]:
def SBL(X, y, alpha_, beta_):
    n_samples = X.shape[0]
    X_normed = X / torch.norm(X, dim=0, keepdim=True)
    
    A_inv = torch.inverse(torch.diag(alpha_) + beta_ * X_normed.T @ X_normed)
    mn = beta_ * A_inv @ X_normed.T @ y
    E = beta_ * torch.sum((y - X_normed @ mn)**2) + (alpha_[:, None] * mn**2).sum()
    
    p =  E - (torch.logdet(A_inv) + n_samples * torch.log(beta_) + torch.sum(torch.log(alpha_))) # we use alpha and lambda since these are bounded
    return p, mn

In [47]:
network = NN(2, [30, 30, 30, 30, 30], 1)
library = Library1D(poly_order=2, diff_order=3) # Library function
estimator = Threshold(0.1) # Sparse estimator 
constraint = LeastSquares() # How to constrain
model = DeepMoD(network, library, estimator, constraint, 12).to(device) # Putting it all in the model

sparsity_scheduler = TrainTestPeriodic(periodicity=50, patience=500, delta=1e-6) # in terms of write iterations
optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.99), amsgrad=True, lr=1e-3) # Defining optimizer

In [48]:
train_SBL(model, X, y, optimizer, sparsity_scheduler, exp_ID='SBL_super_small_threshold_domain', split=0.8, write_iterations=25, max_iterations=25000, delta=1e-3, patience=200) 

  8400  MSE: 1.46e-03  Reg: 3.09e-06  L1: 1.04e+00 Algorithm converged. Writing model to disk.
